In [1]:
import stageemi.dev.decorator_map as dm
import xarray as xr 
import ipywidgets as widg 
import ipyleaflet as ipyl
import datetime as dt
from datetime import datetime, timedelta
import os 
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl,LayersControl
import pandas
import numpy as np
import matplotlib.cm as cm
import matplotlib as mpl

In [2]:
def printf(change):
    print(change)
d = widg.DatePicker()
d.observe(printf,"value")

In [3]:
class interactive_map(widg.HBox):
    def __init__(self,zone,date):
        
        self.m = ipyl.Map(basemap=ipyl.basemaps.OpenTopoMap,center=(43.5,3.5),zoom=7)#(46.5,3),zoom=8)
        # Date 
        date_picker = widg.DatePicker(value=date)
        self._date = date_picker.value 
#         print("on initialise la date",self._date)
        date_picker.observe(self.change_date,"value")
#         print("on observe un changement de date",self._date)
        self.step = 0
        variable_picker = widg.Dropdown(value="WWMF",options=["WWMF","PRECIP","T"])
        variable_picker.observe(self.variable_change,"value")
        self._variable = variable_picker.value
        # Open dataset and mask 
        self.open_file()
        self.zone=zone
        self.get_mask()
        
        # Add other widgets 
        self.legend = widg.Image(layout=widg.Layout(height="430px"))
        self.html1 = HTML('''
                    <h4>Type de temps</h4>
                        Hover over a pixel
                        ''')
        self.html1.layout.margin = '0px 20px 10px 20px'
        
        self.html2 = HTML('''
                    <h4><b>{}</b></h4><h4>wwmf maj<br />agat<br />compas</h4>
                        Hover over a pixel
                        '''.format(self._date.strftime("%Y %m %d %H %M")))
        
        self.html2.layout.margin = '0px 20px 10px 20px'
        # Add controls 
        control1 = WidgetControl(widget=self.html1, position='bottomright')
        self.m.add_control(control1)
        
        control2 = WidgetControl(widget=self.html2, position='bottomleft')
        self.m.add_control(control2)
    
        slider = widg.IntSlider(min=0,max=len(self.da.step)-1,step=1,value=0,description="step")
        slider.observe(self.change_step,'value')
        self.m.add_control(ipyl.WidgetControl(widget=widg.VBox([date_picker,slider,variable_picker]),position="topright"))
        self.render()
        self.m.add_control(LayersControl(position='topleft'))
        super().__init__([self.m,self.legend])
        
    @property 
    def variable(self):
        return self._variable
   
    @variable.setter 
    def variable(self,variable):
        self._variable = variable
        self.open_file()
        self.render()
        
    @property 
    def date(self):
        return self._date
    
    @date.setter
    def date(self,date):
        self._date = date
        self.open_file()
        self.render()    
    
    @property 
    def mask(self):
        return self._mask 
    
    @mask.setter
    def mask(self,mask):
        self._mask = mask 
        if hasattr(self,"da"):
            self.mask_da()
            
    def variable_change(self,change):
        print("quand est-ce qu on rentre ici?",change["new"])
        self.variable = change["new"]
        
    def mask_da(self):
        self.da_masked = (self.mask * self.da).squeeze("id")

        
    def get_mask(self):
        da_mask = xr.open_dataarray("../../GeoData/nc_departement/"+self.zone+".nc")     
        da_mask["latitude"] = da_mask["latitude"].round(5)
        
        self.mask = da_mask
        
    def change_date(self,change):
        self.date = dt.datetime(change["new"].year,
                                change["new"].month,
                               change["new"].day)
#         print(change)
#         print("quand est-ce qu on rentre par la?",self.date)
        
    def open_file(self):
        if self.variable == "WWMF":
            fname = "../../WWMF/%s__PG0PAROME__WWMF__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc"%self.date.strftime("%Y%m%d%H%M%S")
            #fname = "/scratch/labia/lepapeb/models_data/WWMF/%s__PG0PAROME__WWMF__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc"%self.date.strftime("%Y%m%d%H%M%S")
        elif self.variable == "PRECIP":
            fname = "/scratch/labia/lepapeb/models_data/PRECIP/%s__PAROME__PRECIP__EURW1S100______GRILLE____1_48_1__SOL____.nc"%self.date.strftime("%Y%m%d%H%M%S")
        elif self.variable == "T":
            fname = "/scratch/labia/lepapeb/models_data/T/%s__PG1PAROME__T__EURW1S100______GRILLE____0_48_1__HAUTEUR__2__.nc"%self.date.strftime("%Y%m%d%H%M%S")
        else: 
            raise(ValueError("Unknown variable"))
        self.da = xr.open_dataarray(fname)
        self.da['latitude'] = self.da.latitude.round(5)
        if hasattr(self,"mask"):
             self.mask_da()
                
    @dm.gogeojson_wwmf(bins=[1,2,3,5,99])
    def get_step(self):
        return self.da_masked.isel(step=self.step)
    
        
    def update_html(self,feature, **kwargs):
        file_CodesWWMF = '../../utils/CodesWWMF.csv'
        df = pandas.read_csv(file_CodesWWMF,usecols = (0,1,2,3,6,7),sep=',')
        legende_WWMF = df['Legende WWMF'].to_numpy()
        code_WWMF    = df['Code WWMF'].to_numpy()
        legende_W1 = df['Legende W1'].to_numpy()
        code_W1    = df['Code W1'].to_numpy()
        legende_WME = df['Legende WME'].to_numpy()
        code_WME    = df['Code WME'].to_numpy()
#         print(feature)
        valeur_code  = feature['properties']['value']
        indice       = np.where(code_WWMF == valeur_code)
        self.html1.value = '''
            <h4> Type de temps </h4>
            <h4><b>{}<br />{}</h4>
        '''.format(legende_WWMF[indice[0]][0],code_WWMF[indice[0]][0]) 
        
#         print(legende_WWMF[indice[0]][0],legende_W1[indice[0]][0],legende_WME[indice[0]][0])
        
        file_score = '../scores/'+self.zone+'_scores_%s'%self.date.strftime("%Y%m%d")+'0100.csv'
#         print(file_score)
        
        df_score = pandas.read_csv(file_score,sep=',',index_col=0)
                
        new_date = self.date+dt.timedelta(hours=self.step+1)
        
#         print(self.date,len(dir(self.date)),self.step,new_date)
#         print(df_score.head())
        
        WWMF_maj = df_score.loc[new_date.strftime("%Y %m %d %H %M")]['WWMF']        
        W1_maj = df_score.loc[new_date.strftime("%Y %m %d %H %M")]['W1']
        WME_maj = df_score.loc[new_date.strftime("%Y %m %d %H %M")]['WME']
        nb_accord = df_score.loc[new_date.strftime("%Y %m %d %H %M")]['WWMF vs (WME and W1)']

        indice_wwmf = np.where(code_WWMF == WWMF_maj)
        indice_w1 = np.where(code_W1 == W1_maj)
        indice_wme = np.where(code_WME == WME_maj)
        
        if nb_accord < 2:
            self.html2.value = '''
                <h4><b>{}</b></h4><h4>wwmf maj {} ({})<br/>agat {} ({})<br/>compas {} ({})<br/></h4><h4>Desaccord: {}</h4>
            '''.format(new_date.strftime("%Y %m %d %H %M"),WWMF_maj,legende_WWMF[indice_wwmf[0]][0],
                                                       W1_maj,legende_W1[indice_w1[0]][0],
                                                       WME_maj,legende_WME[indice_wme[0]][0],2-nb_accord)
            
        else:
            self.html2.value = '''
                <h4><b>{}</b></h4><h4>wwmf maj {} ({})<br/>agat {} ({})<br/>compas {} ({})</h4>
            '''.format(new_date.strftime("%Y %m %d %H %M"),WWMF_maj,legende_WWMF[indice_wwmf[0]][0],
                                                       W1_maj,legende_W1[indice_w1[0]][0],
                                                       WME_maj,legende_WME[indice_wme[0]][0])
            
    def change_step(self,change):
        self.step = change["new"]
        self.render()
        
    def render(self):
        geo_file,legend_file = self.get_step()
        geojson_layer = ipyl.GeoJSON(data=geo_file,hover_style={"opacity":1},name=self.zone)
        if hasattr(self,"geojson_layer"):
            if self.geojson_layer in self.m.layers:
                self.m.substitute_layer(self.geojson_layer,geojson_layer)
            else: 
                self.m.add_layer(geojson_layer)
        else:
            self.m.add_layer(geojson_layer)
        self.geojson_layer = geojson_layer
        self.geojson_layer.on_hover(self.update_html)
        legend_file.seek(0)
        self.legend.value =legend_file.read() 
        
        

In [8]:
zone="FRH02" #['HG31',FRJ13','FRB05','FRH02','FRK24']
date=dt.datetime(2020,1,26)

m = interactive_map(zone,date)
# m.get_mask("FRJ13")
# m.zone="FRJ13"
# print(m.zone)
m

interactive_map(children=(Map(center=[43.5, 3.5], controls=(ZoomControl(options=['position', 'zoom_in_text', '…